Note: The filepaths in this notebook are no longer valid. This notebook is meant only to be viewed as a reference. All of the code in this notebook has been adapted into [02-data_cleaning.py](./../src/data/02-data_cleaning.py)

# Import and Clean Data

In [56]:
# Import packages
import pandas as pd
import numpy as np
import os

## Function Declarations

In [71]:
def import_demographics(year_dir_path):
    # Find filepath
    for subdir, dirs, files in os.walk(year_dir_path + "Demographics/"):
        for file in files:
            if file.endswith(".csv") & ("DEMO" in file):
                filepath = os.path.join(subdir, file)

    demographics = pd.read_csv(filepath)
    dems = demographics[['SEQN', 'RIAGENDR', 'RIDAGEMN', 'RIDAGEYR', 'RIDEXPRG', 'RIDRETH1']].copy()
    dems.set_axis(
        [
            'id',
            'sex',                  # 1: Male, 2: Female
            'age_months',
            'age_yrs',
            'pregnancy_status',     # 1: Pregnant, 2: Not pregnant, 3: Unknown
            'race'                  
        ], 
        axis=1, 
        inplace=True
    )
    dems = dems.astype({
        "id": "int",
        "sex": "int",
        "race": "int"
    })
    dems.set_index('id', inplace=True)

    return dems

def import_body_data(body_data_dir_path):
    # Find filepath for body data
    for subdir, dirs, files in os.walk(body_data_dir_path + "Examination/"):
        for file in files:
            if file.endswith(".csv") & ("BMX" in file):
                filepath_measurements = os.path.join(subdir, file)
                
    # Import body measurements
    body_measurements = pd.read_csv(filepath_measurements)
    measurements = body_measurements[['SEQN', 'BMXWT', 'BMXHT', 'BMXWAIST']].copy()
    measurements.set_axis(['id', 'weight_kg', 'height_cm', 'waist_circum_cm'], axis=1, inplace=True)
    measurements = measurements.astype({"id": "int"})
    measurements.set_index('id', inplace=True)

    # Drop rows with missing height or weight
    measurements = measurements[measurements['weight_kg'].notna() & measurements['height_cm'].notna()]

    # Find filepath for body data
    for subdir, dirs, files in os.walk(body_data_dir_path + "Questionnaire/"):
        for file in files:
            if file.endswith(".csv") & (("SMQ_" in file) | ("SMQ." in file)):
                filepath_smoker = os.path.join(subdir, file)

    # Import smoker indication
    smoker_indication = pd.read_csv(filepath_smoker)
    smoker = smoker_indication[[
        'SEQN', 
        'SMQ020', # Smoked at least 100 cigarettes in life (1: Yes, 2: No, 7: Refused, 9: Don't know)
        'SMQ040'  # Do you know smoke cigarettes (1: Every day, 2: Some days, 3: Not at all, 7: Refused, 9: Don't know)
    ]].copy()
    smoker.set_axis([
        'id', 
        'has_smoked', 
        'is_smoker'
    ], axis=1, inplace=True)
    smoker = smoker.astype({"id": "int"})
    smoker.set_index('id', inplace=True)

    # Join smoker and body measurements data
    measurements_smoker = pd.merge(measurements, smoker, how='left', on='id')

    return measurements_smoker

def import_fat_pct(fat_pct_dir_path):
    # Find filepath for percent body fat data
    for subdir, dirs, files in os.walk(fat_pct_dir_path + "Examination/"):
        for file in files:
            if file.endswith(".csv") & ("dxx" in file):
                filepath = os.path.join(subdir, file)

    # Import percent body fat data
    fat_percentage = pd.read_csv(filepath)
    fat_pct = fat_percentage[['SEQN', '_MULT_', 'DXDTOPF']]
    fat_pct.set_axis(['id', 'M', 'total_fat_pct'], axis=1, inplace=True)
    fat_pct = fat_pct.astype({"id": "int", "M": "int"})

    # Drop rows with missing data
    # fat_pct = fat_pct[fat_pct['total_fat_pct'].notna()]


    return fat_pct


In [72]:
def import_year_predictors(year_range):
    # Import predictors
    demographics = import_demographics("./../../../Data/NHANES_" + year_range + "/")
    measurements = import_body_data("./../../../Data/NHANES_" + year_range + "/")

    # Join demographic and measurement data
    predictors = pd.merge(measurements, demographics, how='left', on='id')

    return predictors

def import_predictors():
    # Import datasets from each year
    predictors99 = import_year_predictors("1999-2000")
    predictors99['years'] = "1999-2000"
    predictors01 = import_year_predictors("2001-2002")
    predictors01['years'] = "2001-2002"
    predictors03 = import_year_predictors("2003-2004")
    predictors03['years'] = "2003-2004"
    predictors05 = import_year_predictors("2005-2006")
    predictors05['years'] = "2005-2006"

    # Concatenate all datasets into one
    predictors = pd.concat([predictors99, predictors01, predictors03, predictors05], axis=0)

    # CLEANING
    predictors['sex'] = predictors['sex'] - 1

    return predictors

def import_response():
    # Import fat_pct from each year
    response99 = import_fat_pct("./../../../Data/NHANES_1999-2000/")
    response01 = import_fat_pct("./../../../Data/NHANES_2001-2002/")
    response03 = import_fat_pct("./../../../Data/NHANES_2003-2004/")
    response05 = import_fat_pct("./../../../Data/NHANES_2005-2006/")

    # Concatenate all datasets into one
    response = pd.concat([response99, response01, response03, response05], axis=0)

    return response

## Join imputed values onto predictors

In [73]:
predictors = import_predictors()
predictors.to_csv("./data/predictors.csv")
response = import_response()
response.to_csv("./data/response.csv")

# M = 1
responseM1 = response.copy()[response['M'] == 1]
responseM1.set_index('id', inplace=True)
dataM1 = pd.merge(predictors.copy(), responseM1, how="inner", on="id")
dataM1.to_csv("./data/M1_data.csv")

# M = 2
responseM2 = response.copy()[response['M'] == 2]
responseM2.set_index('id', inplace=True)
dataM2 = pd.merge(predictors.copy(), responseM2, how="inner", on="id")
dataM2.to_csv("./data/M2_data.csv")

# M = 3
responseM3 = response.copy()[response['M'] == 3]
responseM3.set_index('id', inplace=True)
dataM3 = pd.merge(predictors.copy(), responseM3, how="inner", on="id")
dataM3.to_csv("./data/M3_data.csv")

# M = 4
responseM4 = response.copy()[response['M'] == 4]
responseM4.set_index('id', inplace=True)
dataM4 = pd.merge(predictors.copy(), responseM4, how="inner", on="id")
dataM4.to_csv("./data/M4_data.csv")

# M = 5
responseM5 = response.copy()[response['M'] == 5]
responseM5.set_index('id', inplace=True)
dataM5 = pd.merge(predictors.copy(), responseM5, how="inner", on="id")
dataM5.to_csv("./data/M5_data.csv")


## Run transformation on smoking columns and re-export datasets

In [74]:
# THIS CODE IS TAKEN FROM "eda.ipynb"
predictors_smoking = predictors.copy()

# Drop 'age_months' since there is no missing data in age_yrs
predictors_smoking.drop(['age_months'], axis=1, inplace=True)

# Construct 'smoker_status' column
predictors_smoking['smoker_status'] = np.nan

# Set 'smoker_status' column to 0 for non-smokers (who have never been smokers)
predictors_smoking.loc[predictors_smoking['has_smoked'] == 2, 'smoker_status'] = 0

# Set 'smoker_status' column to 1 for active smokers
predictors_smoking.loc[(predictors_smoking['is_smoker'] == 1) | (predictors_smoking['is_smoker'] == 2), 'smoker_status'] = 1

# Set 'smoker_status' column to 2 for former smokers
predictors_smoking.loc[predictors_smoking['is_smoker'] == 3, 'smoker_status'] = 2

# Drop 'has_smoked' and 'is_smoker' columns now that they are consolidated into 'smoker_status'
predictors_smoking.drop(['has_smoked', 'is_smoker'], axis=1, inplace=True)

# Reorder columns
cols_reordered = [
    'weight_kg', 
    'height_cm', 
    'sex', 
    'age_yrs', 
    'race', 
    'waist_circum_cm', 
    'smoker_status', 
    'pregnancy_status', 
    'years'
]

In [75]:
# Update predictors dataset and rerun code joining each imputation on the response
predictors = predictors_smoking[cols_reordered]

predictors.to_csv("./data_smokercleaned/predictors.csv")

response.to_csv("./data_smokercleaned/response.csv")

# M = 1
responseM1 = response.copy()[response['M'] == 1]
responseM1.set_index('id', inplace=True)
dataM1 = pd.merge(predictors.copy(), responseM1, how="inner", on="id")
dataM1.to_csv("./data_smokercleaned/M1_data.csv")

# M = 2
responseM2 = response.copy()[response['M'] == 2]
responseM2.set_index('id', inplace=True)
dataM2 = pd.merge(predictors.copy(), responseM2, how="inner", on="id")
dataM2.to_csv("./data_smokercleaned/M2_data.csv")

# M = 3
responseM3 = response.copy()[response['M'] == 3]
responseM3.set_index('id', inplace=True)
dataM3 = pd.merge(predictors.copy(), responseM3, how="inner", on="id")
dataM3.to_csv("./data_smokercleaned/M3_data.csv")

# M = 4
responseM4 = response.copy()[response['M'] == 4]
responseM4.set_index('id', inplace=True)
dataM4 = pd.merge(predictors.copy(), responseM4, how="inner", on="id")
dataM4.to_csv("./data_smokercleaned/M4_data.csv")

# M = 5
responseM5 = response.copy()[response['M'] == 5]
responseM5.set_index('id', inplace=True)
dataM5 = pd.merge(predictors.copy(), responseM5, how="inner", on="id")
dataM5.to_csv("./data_smokercleaned/M5_data.csv")
